In [1]:
import pandas as pd
import numpy as np
import h5py

# load data
df = pd.read_csv (r'data/raw_data/funding_data.csv')

# replace + and - so they are coded as missing
df = df.replace('+', np.NaN, regex=False)
df = df.replace('-', np.NaN, regex=False)

# Export the pandas DataFrame into HDF5
h5File = "data/cleaned_data/funding_data.hdf5";
df.to_hdf(h5File, "/data/cleaned_data");

/srv/conda/envs/notebook/lib/python3.9/site-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['research_category', '2008', '2009', '2009_ARRA', '2010', '2010_ARRA',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2021_Estimated', '2022_Estimated', '2019_US_Mortality_19',
       '2019_US_Prevalence_SE_19'],
      dtype='object')]

  pytables.to_hdf(


In [2]:
# Use pandas again to read data from the hdf5 file to the pandas DataFrame
df = pd.read_hdf(h5File, "/data/cleaned_data");